# Home Assigment


#### Import Libraries

In [21]:
import pandas as pd
import sqlite3

## First Assigment


You need to calculate for each login in logins table (please see below) what is the login level. For example, login id 222 is the first login so its level is 0, its child logins’ level is 1 etc. please write a SQL query that update the login level field.

1. **Creating a Temporary CTE (Common Table Expression)**:
   - I began by creating a temporary CTE called `RecursiveCTE`.

2. **Anchor Member**:
   - In the anchor member of the recursive query, I selected the root logins. These are logins that have no parent login. I set their initial level to 0, representing the top of the hierarchy.

3. **Recursive Member**:
   - In the recursive member of the query, I performed a self-join on the `logins` table and the `RecursiveCTE`. This allowed me to traverse the hierarchical structure of logins.
   - For each iteration, I incremented the login level by 1. This means that for each child login, the level increased by 1 compared to its parent, creating a hierarchy.

4. **Updating the Login Levels**:
   - Finally, I used an `UPDATE` statement to update the `login_level` field in the original `logins` table based on the results obtained from the recursive CTE. 


In [98]:
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

# SQL code for finding login levels
sql_code = """
-- Step 1: Create a temporary table to store the login levels
WITH RecursiveCTE AS (
    -- Step 2: Anchor member - Start with the logins with no parent (level 0)
    SELECT LoginID, 0 AS LoginLevel -- Initialize the level for root logins as 0
    FROM Logins
    WHERE ParentLoginID IS NULL

    UNION ALL

    -- Step 3: Recursive member - Join with the parent logins to calculate levels for children
    SELECT l.LoginID, r.LoginLevel + 1 AS LoginLevel -- Increment level for child logins
    FROM Logins l
    JOIN RecursiveCTE r ON l.ParentLoginID = r.LoginID
)
-- Step 4: Update the login level in the original logins table
UPDATE Logins
SET LoginLevel = (SELECT LoginLevel FROM RecursiveCTE WHERE RecursiveCTE.LoginID = Logins.LoginID);
"""

cursor.executescript(sql_code)
conn.commit()

sql_query = "SELECT * FROM Logins"
df = pd.read_sql_query(sql_query, conn)

conn.close()
df.head(5) 

,LoginID,ParentLoginID,LoginLevel
0,111,222.0,1
1,222,NaN,0
2,333,111.0,2
3,444,222.0,1
4,555,111.0,2


## Second Assigment


•	Please create DWH tables scheme with clustered columnstore index:
	Fact_Money_In – 
•	add Amount_USD field that will reflect the USD equivalent amount of the transaction 
•	the table should include Card and Account money in transactions (still need to be able to define between Card or Account transactions), and also the extended details data

•	Develop ETL process to load Fact_Money_In using SQL.
Please pay attention that you don’t have exchange rate for every day. In this case you should use the last exchange rate exists before the transaction date
Please take in consideration that the number of records is big so you need to build the ETL process to be incremental.




1. **Table Creation**:
   - I made a table named `Fact_Money_In` to store info about money-in transactions, like transaction ID, customer ID, date, currency, amount, company details, and a USD equivalent.

2. **Data Loading and Cleanup**:
   - I brought in data from two sources, checked for errors, and fixed negative amounts.

3. **Integration with More Info**:
   - I combined this data with extra details from `MoneyIn_Extended_Details`, keeping only the valid ones.

4. **Currency Conversion to USD**:
   - Finally, I converted all amounts to USD using exchange rates from the `Currencies` table.



### Create  Fact_Money_In table and load data

In [97]:
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

# SQL code for creating the Fact_Money_In table and loading data
sql_code = """
-- Create the Fact_Money_In table
CREATE TABLE Fact_Money_In (
    TransactionID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    TransactionDate DATE,
    Currency VARCHAR(3),
    Amount DECIMAL(18, 2),
    CompanyID INTEGER,
    CompanyCategory VARCHAR(50),
    UpdateDate DATETIME,
    Amount_USD DECIMAL(18, 2)
);

-- Create a Clustered Columnstore Index
-- (Note: SQLite does not support clustered columnstore indexes so I'm writing it as remark)
-- CREATE CLUSTERED COLUMNSTORE INDEX CCI_Fact_Money_In ON Fact_Money_In;


-- Load data into Fact_Money_In, apply data validation and transformation
INSERT INTO Fact_Money_In (
    TransactionID,
    CustomerID,
    TransactionDate,
    Currency,
    Amount,
    CompanyID,
    CompanyCategory,
    UpdateDate,
    Amount_USD
)
SELECT
    m.TransactionID,
    m.CustomerID,
    m.TransactionDate,
    CASE WHEN m.Currency IN ('USD', 'EUR', 'GBP', 'CAD') THEN m.Currency ELSE 'Other' END,
    CASE WHEN m.Amount >= 0 THEN m.Amount ELSE 0 END,
    m.CompanyID,
    m.CompanyCategory,
    m.UpdateDate,
    0.0
FROM (
    SELECT * FROM Account_MoneyIn
    UNION ALL
    SELECT * FROM Card_MoneyIn
) m;

"""

cursor.executescript(sql_code)
conn.commit()

sql_query = "SELECT * FROM Fact_Money_In"
df = pd.read_sql_query(sql_query, conn)
conn.close()
df.head()  

,TransactionID,CustomerID,TransactionDate,Currency,Amount,CompanyID,CompanyCategory,UpdateDate,Amount_USD
0,4256778,103658,2017-07-20,USD,200.00,1256,Marketplace,None,0
1,4256779,103658,2017-07-21,EUR,30.00,1256,Marketplace,None,0
2,4256780,103658,2017-07-21,USD,150.00,1256,Marketplace,None,0
3,4256781,103658,2017-07-22,EUR,153.00,20156,Freelance,None,0
4,4256782,103700,2017-07-25,GBP,144.06,20156,Freelance,None,0


### Calculate USD Equivalent Amount of Transactions

In [122]:
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

# SQL code to calculate USD Equivalent Amount of Transactions and update Fact_Money_In
sql_code = """
-- Calculate USD Equivalent Amount of Transactions and update Fact_Money_In
UPDATE Fact_Money_In
SET Amount_USD = Amount * COALESCE(
    (SELECT Rate
     FROM Currencies
     WHERE CurrencyFrom = Fact_Money_In.Currency
     AND CurrencyTo = 'USD'
     AND Date = (
         SELECT MAX(Date)
         FROM Currencies
         WHERE CurrencyFrom = Fact_Money_In.Currency
         AND Date <= Fact_Money_In.TransactionDate
     )), 1);
"""

cursor.executescript(sql_code)
conn.commit()

sql_query = "SELECT * FROM Fact_Money_In"
df = pd.read_sql_query(sql_query, conn)

conn.close()
df.head()

,TransactionID,CustomerID,TransactionDate,Currency,Amount,CompanyID,CompanyCategory,UpdateDate,Amount_USD
0,4256778,103658,2017-07-20,USD,200.00,1256,Marketplace,None,200.000000
1,4256779,103658,2017-07-21,EUR,30.00,1256,Marketplace,None,48.000000
2,4256780,103658,2017-07-21,USD,150.00,1256,Marketplace,None,150.000000
3,4256781,103658,2017-07-22,EUR,153.00,20156,Freelance,None,244.800000
4,4256782,103700,2017-07-25,GBP,144.06,20156,Freelance,None,198.673146


### Join MoneyIn_Extended_Details

In [138]:
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

# SQL code to create the Fact_Money_In_extended table
create_table_sql = """
CREATE TABLE IF NOT EXISTS Fact_Money_In_extended (
    TransactionID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    TransactionDate DATE,
    Currency VARCHAR(3),
    Amount DECIMAL(18, 2),
    CompanyID INTEGER,
    CompanyCategory VARCHAR(50),
    UpdateDate DATETIME,
    Amount_USD DECIMAL(18, 2),
    TransactionMethod VARCHAR(15), 
    In_fee FLOAT
);

-- Create a Clustered Columnstore Index
-- (Note: SQLite does not support clustered columnstore indexes so I'm writing it as remark)
-- CREATE CLUSTERED COLUMNSTORE INDEX CCI_Fact_Money_In ON Fact_Money_In;

"""
cursor.execute(create_table_sql)

# SQL code to INSERT INTO Fact_Money_In_extended from the join of Fact_Money_In and MoneyIn_Extended_Details
insert_sql = """
INSERT INTO Fact_Money_In_extended
SELECT f.TransactionID, f.CustomerID, f.TransactionDate, f.Currency, f.Amount, f.CompanyID, f.CompanyCategory, f.UpdateDate, f.Amount_USD, e.TransactionMethod, e.In_fee
FROM Fact_Money_In f
JOIN MoneyIn_Extended_Details e
ON f.TransactionID = e.TransactionID;
"""

cursor.execute(insert_sql)
conn.commit()

sql_query = "SELECT * FROM Fact_Money_In_extended"
df = pd.read_sql_query(sql_query, conn)
cursor.close()
conn.close()
df.head()

,TransactionID,CustomerID,TransactionDate,Currency,Amount,CompanyID,CompanyCategory,UpdateDate,Amount_USD,TransactionMethod,In_fee
0,4256778,103658,2017-07-20,USD,200.00,1256,Marketplace,None,200.000000,Card,0.02
1,4256779,103658,2017-07-21,EUR,30.00,1256,Marketplace,None,48.000000,Card,0.02
2,4256780,103658,2017-07-21,USD,150.00,1256,Marketplace,None,150.000000,Account,0.02
3,4256781,103658,2017-07-22,EUR,153.00,20156,Freelance,None,244.800000,Card,0.05
4,4256782,103700,2017-07-25,GBP,144.06,20156,Freelance,None,198.673146,Account,0.02
